# TD 11 & 12 : Analyse de tweets

    Dans ce TD nous allons étudier et mettre en oeuvre le fonctionnement d'une plateforme d'analyse de données que l'on a nommé InPoDa.

# 1. Diagramme

![Diagramme d'InPoDa](diagramme.png "Diagramme d'InPoDa")

# 2. Structure de données

public_metrics : Statistiques du tweet en question


id : Nombre d'identification du tweet


conversation_id : Nombre d'identification de la conversation


context_annotations : Sujets de la publication


entities : Contient les mentions et les hashtags


author_id : Nombre d'identification de l'auteur


text : Text du tweet


geo : Geolocalisation du tweet 


lang : Langue du tweet


created_at : Date et heure de publication

# 3. Code

## 3.1. Modules

In [123]:
import json as js
import textblob as tb
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import pandas as pd
import tkinter as tk
from copy import deepcopy

## 3.2. Classes

In [2]:
class ScoreList:
    '''
        Classe utilitaire qui permet d'utiliser deux listes en meme temps.
    '''
    def __init__(self, names = [], scores = []):
        self.names = names.copy()
        self.scores = scores.copy()
        
    def index(self, e):
        try:
            return self.names.index(e)
        except:
            return -1

    def fusion_sort(self, g, d):
        '''
            Tri fusion des listes en fonction de la liste self.scores.
        '''
        
        def fusion(g, d):
            m = int((g + d)/2)

            new_slice = ScoreList(self.names[g:d], self.scores[g:d])
            i = g
            j = m
            k = 0
            
            while i < m and j < d:
                if self.scores[i] < self.scores[j]:
                    new_slice.scores[k] = self.scores[j]
                    new_slice.names[k] = self.names[j]
                    j += 1
                else:
                    new_slice.scores[k] = self.scores[i]
                    new_slice.names[k] = self.names[i]
                    i += 1

                k += 1
            while i < m:
                new_slice.scores[k] = self.scores[i]
                new_slice.names[k] = self.names[i]
                i += 1
                k += 1
            while j < d:
                new_slice.scores[k] = self.scores[j]
                new_slice.names[k] = self.names[j]
                j += 1
                k += 1
            
            for i in range(d - g):
                self.scores[g + i] = new_slice.scores[i]
                self.names[g + i] = new_slice.names[i]
        
        if g + 1 == d:
            return
        
        self.fusion_sort(g, int((g+d)/2))
        self.fusion_sort(int((g+d)/2), d)
        fusion(g, d)

class Ind:
    '''
        Représente toute entité comportant les champs Id, Name et Description.
    '''
    def __init__(self, id = None, name = None, description = None):
        self.id = id
        self.name = name
        self.description = description

    def load_from_json(self, raw):
        self.id = raw.get("id")
        self.name = raw.get("name")
        self.description = raw.get("description")

    def to_json(self):
        return {
            "id" : self.id,
            "name" : self.name,
            "description" : self.description
        }

class Hashtag:
    '''
        Représente un hashtag.
    '''
    def __init__(self, tag = "", start = 0, end = 0):
        self.tag = tag
        self.start = start
        self.end = end

    def load_from_json(self, raw):
        self.tag = raw["tag"]
        self.start = raw["start"]
        self.end = raw["end"]
    
    def to_json(self):
        return {
            "start" : self.start,
            "end" : self.end,
            "tag" : self.tag
        }

class Topic:
    '''
        Représente un topic.
    '''
    def __init__(self, domain = Ind(), entity = Ind()):
        self.domain = deepcopy(domain)
        self.entity = deepcopy(entity)

    def load_from_json(self, raw):
        self.domain.load_from_json(raw.get("domain"))
        self.entity.load_from_json(raw.get("entity"))
        
    
    def to_json(self):
        return {
            "domain" : self.domain.to_json(),
            "entity" : self.entity.to_json()
        }

class Tweet:
    '''
            Représente un tweet.
    '''
    def __init__(self, text = "", id = "", author_id = "", hashtags = [], mentions = [], topics = []):
        self.text = text
        self.id = id
        self.author_id = author_id
        self.hashtags = hashtags.copy()
        self.mentions = mentions.copy()
        self.topics = topics.copy()
    
    def load_from_json(self, raw):
        '''
                Charge les propriétés depuis un dictionnaire.
        '''
        self.text = raw["text"]
        self.id = raw["id"]
        self.author_id = raw["author_id"]
        self.hashtags = []
        self.topics = []

        # Chargement des hashtags
        if raw.get("entities") != None:
            for e in raw["entities"]:
                if e == "hashtags":
                    hashtags = raw["entities"][e]
                    for hashtag in hashtags:
                        ha = Hashtag()
                        ha.load_from_json(hashtag)
                        self.hashtags.append(ha)
        
        # Extraction des mentions
        buffer = ""
        reading = False
        for c in self.text:
            if c == '@':
                reading = True
            elif reading and c == ' ':
                reading = False
                self.mentions.append(buffer)
                buffer = ""
            elif reading:
                buffer += c
        
        # Chargement des topics
        context_annotations = raw.get("context_annotations")
        if context_annotations != None:
           for e in context_annotations:
               t = Topic()
               t.load_from_json(e)
               self.topics.append(t)

    
    def to_json(self):
        '''
                Retourne la représentation json de self.
        '''
        return {
            "id" : self.id,
            "author_id" : self.author_id,
            "text" : self.text,
            "hashtags" : [
                h.to_json() for h in self.hashtags
            ],
            "mentions" : self.mentions,
            "context_annotations" : [
                t.to_json() for t in self.topics
            ]
        }
    
    def clean_text(self):
        '''
                Enlève les charactères non alphanumériques de self.text.
        '''
        r = ""
        for c in self.text:
            if c.isalnum() or c == ' ':
                r += c

        self.text = r



## 3.3. Nettoyage des données

In [3]:
def extract_tweets(source):
    '''
            Extrait les tweets d'un fichier source.
    '''
    raw = {}
    tweets = []
    
    with open(source, 'r') as src:
        raw = js.load(src)

    # Chargement des tweets à partir de JSON
    for r in raw:
        t = Tweet()
        t.load_from_json(r)
        tweets.append(t)
    
    return tweets

def clean_tweets(tweets):
    '''
            Retourne un jeu de tweet nettoyé.
    '''
    for tweet in tweets:
        tweet.clean_text()

    return tweets

def convert_to_cleaned(source, target):
    '''
            Convertit le jeu de tweet contenue dans le fichier source et l'écrit dans le
        fichier target.
    '''

    src = extract_tweets(source)
    src = clean_tweets(src)

    src_raw = []
    for tweet in src:
        src_raw.append(tweet.to_json())
    with open(target, 'w') as trg:
        js.dump(src_raw, trg, indent = 4, ensure_ascii = False)
    
convert_to_cleaned("versailles_tweets_100.json", "zone d'atterissage.json")
        

## 3.4. Traitement des données des tweets

In [155]:
def is_positive(t: Tweet):
    '''
        retourne True si le text du tweet est positif, False sinon.
        /!\ Attention : Ne marche bien qu'avec les tweets en anglais /!\
    '''
    blob = tb.TextBlob(t.text)
    return blob.sentiment.polarity > 0


def top_hashtags(collection, k):
    '''
        Retourne les k hashtags les plus utilisés dans la collection de tweet.
    '''
    data = ScoreList()
    
    for tweet in collection:
        for hashtag in tweet.hashtags:
            i = data.index(hashtag.tag)
            if i == -1:
                data.names.append(hashtag.tag)
                data.scores.append(1)
            else:
                data.scores[i] += 1
    
    data.fusion_sort(0, len(data.names))

    return ScoreList(data.names[0:k], data.scores[0:k])

def top_users(collection, k):
    '''
        Retourne les k utilisateurs qui tweetent le plus.
    '''
    data = ScoreList()
    
    for tweet in collection:
        i = data.index(tweet.author_id)
        if i == -1:
            data.names.append(tweet.author_id)
            data.scores.append(1)
        else:
            data.scores[i] += 1
    
    data.fusion_sort(0, len(data.names))

    return ScoreList(data.names[0:k], data.scores[0:k])

def top_mentionned_users(collection, k):
    '''
        Retourne les k utilisateurs les plus mentionnés.
    '''
    data = ScoreList()
    
    for tweet in collection:
        for mention in tweet.mentions:
            i = data.index(mention)
            if i == -1:
                data.names.append(mention)
                data.scores.append(1)
            else:
                data.scores[i] += 1
    
    data.fusion_sort(0, len(data.names))

    return ScoreList(data.names[0:k], data.scores[0:k])

def top_topics(collection, k):
    '''
        Retourne les k topics les plus mentionnés.
    '''
    data = ScoreList()
    
    for tweet in collection:
        for topic in tweet.topics:
            i = data.index(topic.entity.name)
            if i == -1:
                data.names.append(topic.entity.name)
                data.scores.append(1)
            else:
                data.scores[i] += 1
    
    data.fusion_sort(0, len(data.names))

    return ScoreList(data.names[0:k], data.scores[0:k])



def get_user_tweet_count(collection, id):
    '''
        Retourne le nombre de tweets pour l'utilisateur d'id.
    '''
    r = 0
    for t in collection:
        if t.author_id == id:
            r += 1
    return r

def get_hashtag_tweet_count(collection, hashtag):
    '''
        Retourne le nombre de tweets contenant le hashtag.
    '''
    r = 0
    for t in collection:
        for h in t.hashtags:
            if h.tag == hashtag.tag:
                r += 1
    return r

def get_topic_tweet_count(collection, topic):
    '''
        Retourne le nombre de tweets contenant le topic.
    '''
    r = 0
    for t in collection:
        for to in t.topics:
            if to.entity.name == topic.entity.name:
                r += 1
    return r

def get_user_tweets(collection, author_id):
    '''
        Retourne l'intégralité des tweets de l'utilisateur.
    '''
    tweets = []
    for t in collection:
        if t.author_id == author_id:
            tweets.append(t)
    return tweets

def get_tweets_mentioning(collection, user_id):
    '''
        Retourne l'intégralité des tweets mentionnant l'utilisateur.
    '''
    tweets = []
    for t in collection:
        for m in t.mentions:
            if m == user_id:
                tweets.append(t)
    return tweets

def get_users_mentioning(collection, hashtag):
    '''
        Retourne l'intégralité des utilisateurs mentionnant le hashtag.
    '''
    users = []
    for t in collection:
        for h in t.hashtags:
            if h.tag == hashtag.tag:
                users.append(t.author_id)
    return users

def get_users_mentioned_by(collection, author_id):
    '''
        Retourne l'intégralité des utilisateurs mentionnés par l'utilisatuer d'author_id.
    '''
    users = []
    for t in collection:
        if t.author_id == author_id:
            for m in t.mentions:
                users.append(m)
                
    return users




## 3.5. Affichage

### 3.5.1. Récupération des tweets

In [5]:
tweets = extract_tweets("versailles_tweets_100.json")


### 3.5.2. Affichage Top K Hashtags et nombre de publications par hashtag

In [141]:
def show_top_hashtags(k):    
    top = top_hashtags(tweets, k)

    fig = plt.Figure(figsize = (10, 5), dpi = 100)
    sbp = fig.add_subplot(111)
    
    sbp.bar(top.names, top.scores)

    return (fig, "Top Hashtags")


### 3.5.3. Affichage Top K Utilisateurs

In [142]:
def show_top_users(k):    
    top = top_users(tweets, k)

    fig = plt.Figure(figsize = (10, 5), dpi = 100)
    sbp = fig.add_subplot(111)
    
    sbp.bar(top.names, top.scores)

    return (fig, "Top Users")

### 3.5.4. Affichage Top K Utilisateurs mentionnés

In [146]:
def show_top_mentionned_users(k):    
    top = top_mentionned_users(tweets, k)

    fig = plt.Figure(figsize = (10, 5), dpi = 100)
    sbp = fig.add_subplot(111)  
    
    sbp.bar(top.names, top.scores)

    return (fig, "Top Mentionned Users")


### 3.5.5. Affichage Top K Topics et nombre de publications par topic

In [149]:
def show_top_topics(k):    
    top = top_topics(tweets, k)

    fig = plt.Figure(figsize = (10, 5), dpi = 100)
    sbp = fig.add_subplot(111)
    
    sbp.bar(top.names, top.scores)
    return (fig, "Top Topics")

### 3.5.6. Affichage du nombre de publication par utilisateur

In [150]:
def show_users_tweet_count():    

    users = []
    tweet_counts = []

    for t in tweets:
        users.append(t.author_id)
        tweet_counts.append(get_user_tweet_count(tweets, t.author_id))

    fig = plt.Figure(figsize = (10, 5), dpi = 111)
    sbp = fig.add_subplot(111)
    sbp.bar(users, tweet_counts)
    return (fig, "Tweet count for each user")


### 3.5.7. Application

In [165]:
current_pan = None

def command_test(root):
    t = tk.Label(root, text = "Command Test")
    
    return t

def transition(target, c, r):
    global current_pan
    
    current_pan.grid_forget()
    current_pan = target
    current_pan.grid(column = c, row = r, sticky = "n")

def  welcome(root):
    accueil = tk.Frame(root, relief = "ridge", borderwidth = 8)
    titre = tk.Label(accueil, text = "Accueil", font = "Arial 48", bg = "#DDDDDD", width = 15)
    titre.grid(column = 0, row = 0)

    p = tk.Label(accueil, text = "InPoDa propose un ensemble des opérations de traitement de données y compris:\n \
    • Identification de l’auteur de la publication\n \
    • Extraction de la liste de hashtags de la publication\n \
    • Extraction de la liste des utilisateurs mentionnés dans la publication\n \
    • Analyse de sentiment de la publication (le sentiment peut être positif ou bien négatif).\n \
    • Identification du/des topics de la publication\n \
 D’autres opérations d’analyse de données sont également proposés par InPoDa :\n \
    • Top K hashtags (k est un paramètre passé par l’utilisateur)\n \
    • Top K utilisateurs\n \
    • Top K utilisateurs mentionnés\n \
    • Top K topics\n \
    • Le nombre de publications par utilisateur\n \
    • Le nombre de publications par hashtag\n \
    • Le nombre de publications par topic\n \
    • L’ensemble de tweets d’un utilisateur spécifique\n \
    • L’ensemble de tweets mentionnant un utilisateur spécifique\n \
    • Les utilisateurs mentionnant un hashtag spécifique\n \
    • Les utilisateurs mentionnés par un utilisateur spécifique", justify = "left")
    p.grid(column = 0, row = 1)
    
    return accueil

def matplotlib_pan(root, mpl_f):
    fig, title_text = mpl_f
    
    f = tk.Frame(root, relief = "ridge", borderwidth = 8)

    title = tk.Label(f, text = title_text, font = "Arial 48", bg = "#DDDDDD")
    title.grid(column = 0, row = 0)

    canvas = FigureCanvasTkAgg(fig, master = f)
    canvas.get_tk_widget().grid(column = 0, row = 1)    
    
    return f

def top_buttons(root):
    graph_list = tk.Frame(root, relief = "ridge", borderwidth = 8)

    buttons = [
        tk.Button(graph_list, text = "Accueil", relief = "flat", fg = "blue", font = "Arial 18 underline",
                  command = lambda: transition(welcome(root), 1, 0)),
        tk.Button(graph_list, text = "Top Hashtags", relief = "flat", fg = "blue", font = "Arial 18 underline",
                  command = lambda: transition(matplotlib_pan(root, show_top_hashtags(10)), 1, 0)),
        tk.Button(graph_list, text = "Top Users", relief = "flat", fg = "blue", font = "Arial 18 underline",
                  command = lambda: transition(matplotlib_pan(root, show_top_users(10)), 1, 0)),
        tk.Button(graph_list, text = "Top Mentions", relief = "flat", fg = "blue", font = "Arial 18 underline",
                  command = lambda: transition(matplotlib_pan(root, show_top_mentionned_users(10)), 1, 0)),
        tk.Button(graph_list, text = "Top Topics", relief = "flat", fg = "blue", font = "Arial 18 underline",
                  command = lambda: transition(matplotlib_pan(root, show_top_topics(10)), 1, 0)),
        tk.Button(graph_list, text = "Tweets per user", relief = "flat", fg = "blue", font = "Arial 18 underline",
                  command = lambda: transition(matplotlib_pan(root, show_users_tweet_count()), 1, 0))
    ]
    
    for i in range(len(buttons)):
        buttons[i].grid(column = 0, row = i, sticky = 'w')

    return graph_list

def main():
    global current_pan
    
    root = tk.Tk()
    root.configure(bg = "#AAAAAA")
    #root.geometry("800x800")
    root.title("InPoDa")
    
    current_pan = welcome(root)
    g = top_buttons(root)

    g.grid(column = 0, row = 0, sticky = "n")
    current_pan.grid(column = 1, row = 0, sticky = "n")
    
    root.mainloop()

if __name__ == "__main__":
    main()